In [1]:
#import required functions
import warnings
warnings.filterwarnings("ignore")
import math
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pylab as plt
import numpy as np
from IPython.display import Image
import matplotlib as matplotlib
import datetime
from pandas import DataFrame
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from boto.mws.connection import MWSConnection
import mws
from bs4 import BeautifulSoup
import mechanize
import urllib.request
import http.cookiejar as cookielib
import requests
from pprint import pprint
import random

In [2]:
filenamefinal= '/Users/JoyThompson/Desktop/AMAZONPRODSCRAPERfinal.xlsx'
filenamefinal2= '/Users/JoyThompson/Desktop/AMAZONPRODSCRAPERCOMBINED.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')

filename =  '/Users/JoyThompson/Desktop/search terms.xlsx'
termpd = pd.read_excel(filename, sheetname='terms')
termpd=termpd.groupby('Customer Search Term')['Impressions', 'Clicks', 'Spend'].sum().reset_index()
termpd=termpd.set_index('Customer Search Term')

skupd = pd.read_excel(filename, sheetname='skus')
for sku in skupd.index:
    match=skupd.loc[sku, 'Title']
    skupd.loc[sku, 'Product Name']=str(match)
skupdprod=skupd.set_index('Product Name')
skupdsku=skupd.set_index('SKU')

In [3]:
agentlist = [ 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)']

In [4]:
def getaztext(term, agentlist, page): 
    
    url='https://www.amazon.com/dp/s?k=' + term + '&page=' + str(page)
    useragent = random.choice(agentlist)
    headers = {'authority': 'www.amazon.com', 'pragma': 'no-cache',  'cache-control': 'no-cache',    'dnt': '1', 'upgrade-insecure-requests': '1',     'user-agent': useragent,
        'sec-fetch-site': 'none', 'sec-fetch-mode': 'navigate', 'sec-fetch-dest': 'document',     'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8'}
    response =urllib.request.Request(url, headers=headers)
    try:
        result = urllib.request.urlopen(response)
        resulttext = result.read()
        soup = BeautifulSoup(resulttext)
        
        for script in soup(["script", "style"]):
            script.extract()    
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
    
    except:
        pass   
        text='star'
    return text

In [5]:
def getrankings(term, agentlist, skupdprod, skuoutpd):
    dfterm=pd.DataFrame(columns=['Product','SKU' ,'Ranking'])
    
    count=0
    for page in range(1,7):
        text = getaztext(term, agentlist, page)
        items=text[text.find("Amazon's\nChoice"):text.find('Go back to filtering menu')]
        allprod=items.split('stars')
        
        for prod in allprod:
            if len(prod.split('\n')) >3:
                count=count+1
                prod=str(prod.split('\n')[-2])
           
                if prod in skupdprod.index:
                    sku=skupdprod.loc[prod, 'SKU']
            
                    if prod not in dfterm['Product']:
                        dfterm=dfterm.append({'Product':prod, 'SKU': sku, 'Ranking': count}, ignore_index=True)
                        skuoutpd=skuoutpd.append({'Term':term, 'Product':prod, 'SKU': sku, 'Ranking': count}, ignore_index=True)
                    
    return dfterm, skuoutpd

In [6]:
skuoutpd=pd.DataFrame(columns=['Term','SKU', 'Product', 'Ranking'])
for term in termpd.index:
    term=term.replace(' ' , '+')
    dfterm, skuoutpd=getrankings(term, agentlist, skupdprod, skuoutpd)
    sheetterm=term[0:30]
    dfterm.to_excel(writer, sheet_name=sheetterm)
writer.save()

In [7]:
termout=skuoutpd.groupby('Term').sum().reset_index()
termout=termout.set_index('Term')
collist=['Product Name']
for term in termout.index:
    collist.append(term)

finalout=pd.DataFrame(index=skupdsku.index, columns=collist)

for sku in finalout.index:
    name=skupdsku.loc[sku,'Product Name']
    finalout.loc[sku, 'Product Name']=name
    
for ind in skuoutpd.index:
    sku=skuoutpd.loc[ind, 'SKU']
    term=skuoutpd.loc[ind, 'Term']
    rank=skuoutpd.loc[ind, 'Ranking']
    finalout.loc[sku, term]=rank

In [8]:
filenamefinal2= '/Users/JoyThompson/Desktop/Amazon Organic Rank Dammit Dolls Search Terms.xlsx'
writer = pd.ExcelWriter(filenamefinal2, engine='xlsxwriter')
finalout.to_excel(writer, sheet_name='Ranking by SKU and Search Term')
writer.save()